In [2]:
!pip -q install evaluate accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 12.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 9.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 3.7 MB/s eta 0:00:00


In [3]:
from typing import List
import numpy as np
import torch
import evaluate
from sklearn.model_selection import train_test_split

In [4]:
import nltk
nltk.download('treebank')

[nltk_data] Downloading package treebank to /root/nltk_data...
[nltk_data]   Unzipping corpora/treebank.zip.


True

## 1. Load pre-trained checkpoints

In [5]:
from transformers import AutoTokenizer, AutoModelForTokenClassification

model_name = "QCRI/bert-base-multilingual-cased-pos-english"

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/2.12k [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/996k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/712M [00:00<?, ?B/s]

Some weights of the model checkpoint at QCRI/bert-base-multilingual-cased-pos-english were not used when initializing BertForTokenClassification: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [6]:
model

BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, e

## 2. Data Preparation

### 2.1 Load data from nltk corpus

In [7]:
tagged_sentences = nltk.corpus.treebank.tagged_sents()

print("Number of samples:", len(tagged_sentences))

Number of samples: 3914


In [8]:
sentences, sentence_tags =[], []
for tagged_sentence in tagged_sentences:
    sentence, tags = zip(*tagged_sentence)
    sentences.append([word.lower() for word in sentence])
    sentence_tags.append([tag for tag in tags])

### 2.2 Load dictionary to encode label from model config

In [9]:
from collections import defaultdict

label2id = defaultdict(int, model.config.label2id)
id2label = {id: label for label, id in label2id.items()}

In [10]:
label2id

defaultdict(int,
            {'#': 7,
             '$': 6,
             "''": 5,
             ',': 2,
             '-LRB-': 17,
             '-RRB-': 32,
             '.': 4,
             ':': 3,
             'CC': 8,
             'CD': 9,
             'DT': 10,
             'EX': 11,
             'FW': 12,
             'IN': 13,
             'JJ': 14,
             'JJR': 15,
             'JJS': 16,
             'LS': 18,
             'MD': 19,
             'NN': 20,
             'NNP': 21,
             'NNPS': 22,
             'NNS': 23,
             'O': 0,
             'PDT': 24,
             'POS': 25,
             'PRP': 26,
             'PRP$': 27,
             'RB': 28,
             'RBR': 29,
             'RBS': 30,
             'RP': 31,
             'SYM': 33,
             'TO': 34,
             'UH': 35,
             'VB': 36,
             'VBD': 37,
             'VBG': 38,
             'VBN': 39,
             'VBP': 40,
             'VBZ': 41,
             'WDT': 42,
      

In [11]:
len(label2id)

46

### 2.3 Split dataset into train, val and test set

In [12]:
train_sentences, test_sentences, train_tags, test_tags = train_test_split(
    sentences,
    sentence_tags,
    test_size=0.3
)

In [13]:
valid_sentences, test_sentences, valid_tags, test_tags = train_test_split(
    test_sentences,
    test_tags,
    test_size=0.5
)

### 2.4 Build dataset

In [14]:
from torch.utils.data import Dataset

MAX_LEN = max([len(sentence) for sentence in train_sentences])

class PosTagging_Dataset(Dataset):
    def __init__(self,
                 sentences: List[List[str]],
                 tags: List[List[str]],
                 tokenizer,
                 label2id,
                 max_len=MAX_LEN
                ):
        super().__init__()
        self.sentences = sentences
        self.tags = tags
        self.max_len = max_len
        self.tokenizer = tokenizer
        self.label2id = label2id

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, idx):
        input_token = self.sentences[idx]
        label_token = self.tags[idx]

        input_token = self.tokenizer.convert_tokens_to_ids(input_token)
        attention_mask = [1] * len(input_token)
        labels = [self.label2id[token] for token in label_token]

        return {
            "input_ids": self.pad_and_truncate(input_token, pad_id=self.tokenizer.pad_token_id),
            "labels": self.pad_and_truncate(labels, pad_id=label2id["0"]),
            "attention_mask": self.pad_and_truncate(attention_mask, pad_id=0)
        }

    def pad_and_truncate(self, inputs: List[int], pad_id: int):
        if len(inputs) < self.max_len:
            padded_inputs = inputs + [pad_id] * (self.max_len - len(inputs))
        else:
            padded_inputs = inputs[:self.max_len]
        return torch.as_tensor(padded_inputs)

### 2.5 Dataset loader

In [15]:
train_dataset = PosTagging_Dataset(train_sentences, train_tags, tokenizer, label2id)
val_dataset = PosTagging_Dataset(valid_sentences, valid_tags, tokenizer, label2id)
test_dataset = PosTagging_Dataset(test_sentences, test_tags, tokenizer, label2id)

## 3. Train model

In [16]:
accuracy = evaluate.load("accuracy")

ignore_label = len(label2id)

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    mask = labels != ignore_label
    predictions = np.argmax(predictions, axis=-1)
    return accuracy.compute(predictions=predictions[mask], references=labels[mask])

In [17]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="out_dir",
    learning_rate=1e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    tokenizer = tokenizer,
    compute_metrics=compute_metrics,
)

In [18]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.044003,0.987358
2,No log,0.035222,0.989684
3,0.145200,0.031600,0.990709
4,0.145200,0.029464,0.991331
5,0.145200,0.028457,0.991608
6,0.029700,0.027550,0.991859
7,0.029700,0.026936,0.992067
8,0.029700,0.026884,0.992142
9,0.023700,0.026680,0.992155
10,0.023700,0.026557,0.992224


TrainOutput(global_step=1720, training_loss=0.060516599167224974, metrics={'train_runtime': 1775.7326, 'train_samples_per_second': 15.425, 'train_steps_per_second': 0.969, 'total_flos': 3789641345256360.0, 'train_loss': 0.060516599167224974, 'epoch': 10.0})

In [ ]:
trainer.train()

You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.051634,0.985287
2,No log,0.041280,0.987875
3,0.150500,0.037712,0.988907
4,0.150500,0.035710,0.989532
5,0.150500,0.034171,0.989985
6,0.030800,0.033480,0.990071
7,0.030800,0.033291,0.990371
8,0.030800,0.032909,0.990437
9,0.024600,0.032835,0.990484
10,0.024600,0.032753,0.990457


TrainOutput(global_step=1720, training_loss=0.06279984684877618, metrics={'train_runtime': 807.5753, 'train_samples_per_second': 33.916, 'train_steps_per_second': 2.13, 'total_flos': 3579882599208960.0, 'train_loss': 0.06279984684877618, 'epoch': 10.0})

## 4. Test model

In [19]:
trainer.evaluate(test_dataset)

{'eval_loss': 0.0303813386708498,
 'eval_accuracy': 0.9913397093154604,
 'eval_runtime': 11.421,
 'eval_samples_per_second': 51.484,
 'eval_steps_per_second': 3.24,
 'epoch': 10.0}

## 5. Inference

In [21]:
test_sentence = "We are exploring the topic of deep learning"

In [32]:
input = torch.as_tensor([tokenizer.convert_tokens_to_ids(test_sentence.split())])

In [33]:
input

tensor([[12865, 10301,   100, 10105, 57680, 10108, 26591, 26901]])

In [34]:
input = input.to("cuda")

In [35]:
outputs = model(input)
outputs.logits.shape

torch.Size([1, 8, 46])

In [36]:
_, preds = torch.max(outputs.logits, -1)
preds = preds[0].cpu().numpy()

In [37]:
pred_tags = ""

for pred in preds:
    pred_tags += id2label[pred] + " "

pred_tags

'PRP VBP RB DT NN IN JJ NN '